In [130]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from noworkflow.now.models.cleaning.history import *
import numpy as np
from pprint import pprint

In [131]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


### Adapted functions
get_precedent() is an adaptation of get_cells()
find_types()  is an adaptation of find_cells()

In [132]:

def get_precedent(evaluations, options=None):
    """Return code components that contribute to the creation of a given evaluation"""
    result = copy(evaluations)
    querier = DependencyQuerier(options)
    visited = set()
    code_component_op = {}
    operations = set()
    while result:
        nodes_to_visit, visited, found = querier.navigate_dependencies(result, visited=visited)
        result = []
        component_to_visit = []
        new_ops = set()
        # Extract operations from visited evaluations
        for context in visited:
            evaluation = context.evaluation

            # Evaluation operations
            component = evaluation.code_component
            operation = find_types(component, code_component_op)
            if operation and operation not in operations:
                new_ops.add(operation)
                operations.add(operation)
        
        # Expand dependencies to all evaluations of new operations
        for operation in new_ops:
            if operation.this_block:
                result.extend(operation.this_block.recursive_evaluations())

    return operations

In [156]:
def find_types(code_component, found=None):
    """Find specific component typesfrom code_components"""
    found = found or {}
    if code_component in found:
        pass
    elif code_component.type in ('call', 'name'):
        found[code_component] = code_component
    elif not code_component.container_id:
        found[code_component] = None
    else:
        found[code_component] = find_cell(code_component.container.this_component, found=found)
    return found[code_component]

### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [135]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

In [136]:
var_a = 10

In [137]:
var_b = -100

In [138]:
rme_err = mae(var_a, var_b)

In [139]:
var_c = 4869

In [140]:
var_d = var_c * 234
var_c = var_d / 3

In [141]:
var_e = mae(var_c, var_b)

In [142]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, var_e))

Activation(id=145887, name=now_variable, start_checkpoint=6762.187496746, code_block_id=-1)
[now_variable(func), 'var_final'(argument), rmse(var_c, var_e)(argument)] - []
Evaluation(id=145873, checkpoint=6762.187468516, code_component_id=9586, activation_id=145867, repr=100.0)


### New functions test

In [158]:
evals = get_evaluation(145887)
code_comp_list = get_precedent([evals])

In [161]:
code_comp_list

{code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1027, '<class ''str''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 1028, '<class ''numpy.ufunc''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 263, '<class ''NoneType''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 264, '<class ''type''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 482, '<class ''function''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 483, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 9433, '<ipython-input-135-35e08e7bfa73>', 'cell', 'w', 1, 0, 12, 0, nil).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a605, 9481, 'np.mean((x - y))', 'call', 'r', 8, 11, 8, 27, 9469).,
 code_component(6bf7815e-aa57-4896-bc65-01f0d336a60

### Outputing the dependencies of a variable

print_precedents() trying to output a cleaner output from get_precedent() results

In [174]:
def print_precedents(evals):
    code_componenents = get_precedent([evals])
    
    op_dict = {}
    for i in reversed(list(code_comp_list)):
        code_comp = i
        obj_eval = Evaluation((i.trial_id, i.id))
        obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))

        if  ((code_comp.type == 'call') and (code_comp.mode == 'r')) or \
            (code_comp.type == 'name' and code_comp.mode == 'w'): 
                op_dict[round(float(obj_eval.checkpoint), 6)] = [round(float(obj_act.start_checkpoint),4), \
                round(float(obj_eval.checkpoint), 4), code_comp.name, code_comp.type, code_comp.mode]  
    
    return op_dict

In [183]:
print_precedents(evals)

{39.815708: [39.8153, 39.8157, 'mae(var_c, var_b)', 'call', 'r'],
 39.813027: [38.143, 39.813, 'np.mean((x - y)**2)', 'call', 'r'],
 39.808154: [39.8078, 39.8082, 'np.mean((x - y))', 'call', 'r'],
 39.81347: [39.8131, 39.8135, 'var_b', 'name', 'w'],
 39.81604: [38.143, 39.816, 'var_final', 'name', 'w'],
 39.815665: [39.8153, 39.8157, 'var_e', 'name', 'w'],
 39.817431: [38.143, 39.8174, 'rmse(var_c, var_e)', 'call', 'r'],
 39.811567: [38.143, 39.8116, 'np.sqrt(np.mean((x - y)**2))', 'call', 'r'],
 39.815248: [38.143, 39.8152, 'var_c', 'name', 'w'],
 39.81534: [39.8153, 39.8153, 'var_d', 'name', 'w'],
 39.816052: [38.143,
  39.8161,
  "now_variable('var_final', rmse(var_c, var_e))",
  'call',
  'r'],
 39.813351: [39.8131, 39.8134, 'var_a', 'name', 'w'],
 39.815494: [39.8153, 39.8155, 'var_c', 'name', 'w']}